a)Ignorar las primeras celdas ya que este notebook fue desarrollado en google colab. Se tienen aproximadamente 90000 ejemplos para entrenar y cerca de 12000 para testear.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving train_Q-A.csv to train_Q-A.csv


In [0]:
import pandas as pd
df_train = pd.read_csv('train_Q-A.csv')
df_train.dropna(inplace=True)

In [3]:
from google.colab import files
uploaded = files.upload()

Saving test_Q.csv to test_Q.csv


In [4]:

df_test = pd.read_csv('test_Q.csv')
df_test.head()
df_test.shape


(11873, 2)

b) Se deben tokenizar las preguntas y respuestas para su posterior codificación, ya que el input de la red neuronal deben ser enteros y no strings.

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize

train_questions = [word_tokenize(sentence.lower()) for sentence in df_train["question"]] #or processing
test_questions = [word_tokenize(sentence.lower()) for sentence in df_test["question"]]
train_answers = [word_tokenize(sentence) for sentence in df_train["answer"]]

In [7]:
train_questions[0]

['when', 'did', 'beyonce', 'start', 'becoming', 'popular', '?']

c)

In [8]:
#answers
vocab_answer = set()
for sentence in train_answers:
    for word in sentence:
        vocab_answer.add(word)
vocab_answer = ["#end"]+ list(vocab_answer)
print('posibles palabras para respuestas :', len(vocab_answer))
vocabA_indices = {c: i for i, c in enumerate(vocab_answer)}
indices_vocabA = {i: c for i, c in enumerate(vocab_answer)}


#questions_train
vocab_questions_train = set()
for sentence in train_questions:
    for word in sentence:
        vocab_questions_train.add(word)
vocab_questions_train = ["#end"]+ list(vocab_questions_train)
print('posibles palabras para preguntas entrenamiento :', len(vocab_questions_train))
vocabQ_indices_train = {c: i for i, c in enumerate(vocab_questions_train)}
indices_vocabQ_train = {i: c for i, c in enumerate(vocab_questions_train)}

#questions_test
vocab_questions_test = set()
for sentence in test_questions:
    for word in sentence:
        vocab_questions_test.add(word)
vocab_questions_test = ["#end"]+ list(vocab_questions_test)
print('posibles palabras para preguntas test :', len(vocab_questions_test))
vocabQ_indices_test = {c: i for i, c in enumerate(vocab_questions_test)}
indices_vocabQ_test = {i: c for i, c in enumerate(vocab_questions_test)}

posibles palabras para respuestas : 47423
posibles palabras para preguntas entrenamiento : 39482
posibles palabras para preguntas test : 10322


In [116]:
print(vocab_answer[1])
print(vocabA_indices)
print(indices_vocabA)

UB2
{'#end': 0, 'UB2': 1, 'Efendi': 2, 'Opening': 3, 'computed': 4, 'Federal': 5, 'al-shifa': 6, 'Zemeckis': 7, 'humors': 8, '2018': 9, 'VBR': 10, '9.0': 11, 'token': 12, 'Bolshevist': 13, '3.76': 14, 'Barony': 15, 'DMCA': 16, 'Career': 17, 'NGC': 18, 'Vince': 19, 'Plank': 20, 'starvation': 21, 'Tombazis': 22, 'blinking': 23, 'richness': 24, 'Kingpin': 25, '1758': 26, 'tapes': 27, 'analog-only': 28, 'Theories': 29, 'Nationals': 30, '1774': 31, '1,000-3,000': 32, 'considerations': 33, 'DAT': 34, 'Telomeres': 35, '1,250,000': 36, 'panel': 37, 'kidney': 38, 'Near-infrared': 39, 'Keynes': 40, 'Africa-Presbyterian': 41, 'Empirical': 42, 'PNC': 43, 'rainfall': 44, 'Paco': 45, '1888': 46, 'thirty-two': 47, 'lehenga': 48, 'attached': 49, 'government-run': 50, 'DanceSport': 51, 'Henan': 52, 'Sephardim': 53, 'Them': 54, 'tenth': 55, 'pedantry': 56, '5500': 57, 'MMTS': 58, 'Obeidat': 59, 'Tommaso': 60, 'Żywny': 61, 'shops': 62, '1814': 63, 'exhibit': 64, 'Darod': 65, 'Cerberus': 66, 'exceptions':

d)

In [0]:
X_answers = [[vocabA_indices[palabra] for palabra in sentence] for sentence in train_answers]
Xtrain_question = [[vocabQ_indices_train[palabra] for palabra in sentence] for sentence in train_questions]
Xtest_question = [[vocabQ_indices_test[palabra] for palabra in sentence] for sentence in test_questions]

In [10]:

vocabQ_indices_train["#end"]

0

In [11]:
import numpy as np
from keras.preprocessing import sequence

max_input_length_train = np.max(list(map(len,train_questions)))
max_input_length_test = np.max(list(map(len,test_questions)))

if (max_input_length_train>max_input_length_test): max_input_length = max_input_length_train 
else: max_input_length=max_input_length_test

max_output_length = np.max(list(map(len,train_answers)))+1


Xtrain_question_padded = sequence.pad_sequences(Xtrain_question,maxlen=max_input_length,padding='post',value=0)
Xtest_question_padded = sequence.pad_sequences(Xtest_question,maxlen=max_input_length,padding='post',value=0)
X_answers_padded = sequence.pad_sequences(X_answers,maxlen=max_output_length,padding='post',value=vocabA_indices["#end"])

Using TensorFlow backend.


In [12]:
Xtrain_question_padded[0]

array([24505, 32226,  3137, 37538, 10919, 26075, 15359,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0], dtype=int32)

In [0]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=22)

e)

In [0]:
#Encoder-Decoder modelo
from keras.layers import Input,RepeatVector,TimeDistributed,Dense,Embedding,Flatten,Activation,Permute,Lambda
from keras.models import Model
from keras import backend as K
from keras import regularizers

lenght_output = max_output_length
hidden_dim = 128

In [0]:
from keras.layers import CuDNNGRU,GRU,CuDNNLSTM, RNN, Dropout

embedding_vector = 256
encoder_input = Input(shape=(max_input_length,))
embedded = Embedding(input_dim=len(vocabQ_indices_train),output_dim=embedding_vector,input_length=max_input_length)(encoder_input)
embedded = Dropout(0.5)(embedded)
encoder = CuDNNLSTM(hidden_dim, return_sequences=True)(embedded)

In [0]:
# compute T' importance for each step T
attention = TimeDistributed(Dense(max_output_length, activation='relu'))(encoder)
#softmax a las antenciones sobre todo T
attention = Permute([2, 1])(attention)
attention = Activation('softmax')(attention) 
attention = Permute([2, 1])(attention)

In [0]:
# apply the attention to encoder
def attention_multiply(vects):
    encoder, attention = vects
    return K.batch_dot(attention,encoder, axes=1)
sent_representation = Lambda(attention_multiply)([encoder, attention])
sent_representation = Dropout(0.5)(sent_representation)
decoder = CuDNNLSTM(hidden_dim, return_sequences=True)(sent_representation)
probabilities = TimeDistributed(Dense(len(vocab_answer), activation="softmax"))(decoder)

In [310]:
from keras import optimizers
from keras import metrics

model = Model(encoder_input,probabilities)
#0.0045
adam = optimizers.Adam(lr=0.0035)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_21 (Embedding)        (None, 60, 256)      10107392    input_21[0][0]                   
__________________________________________________________________________________________________
dropout_41 (Dropout)            (None, 60, 256)      0           embedding_21[0][0]               
__________________________________________________________________________________________________
cu_dnnlstm_41 (CuDNNLSTM)       (None, 60, 128)      197632      dropout_41[0][0]                 
__________________________________________________________________________________________________
time_distr

Se tiene que las dimensiones de entrada del modelo corresponden a "None" y 60, el none se debe a que es posible pasar todas las "preguntas" que se quiera y de largo 60. Luego se reduce la dimensionalidad con una capa embedding cuyo output es entregado a una capa oculta del tipo LSTM. Una vez codificada la secuencia de entrada se aplica el módulo de atención utilizando una capa timeDistributed debido a que el problema es del tipo seq2seq. Luego para el decoder se utiliza otra capa LSTM cuyo output es entregado a una ultima capa timeDistributed con función de activación softmax para poder predecir una unica clase.

f) Por temas de recursos se decide entrenar con 40000 ejemplos, los cuales se dividen en 32000 para el entrenamiento y 8000 de validación. Cabe destacar que el val_loss tiende a incrementar, por lo que se intuye que faltó entrenamiento (que obviamente traerá consecuencias).

In [311]:
X_answers_padded_reshaped = X_answers_padded.reshape(X_answers_padded.shape[0],X_answers_padded.shape[1],1)
print(X_answers_padded_reshaped.shape)
model.fit(Xtrain_question_padded[40000:80000],X_answers_padded_reshaped[40000:80000],epochs=10,batch_size=32,validation_split=0.2)

(86821, 47, 1)
Train on 32000 samples, validate on 8000 samples
Epoch 1/10
32000/32000 [==============================] - 210s 7ms/step - loss: 0.6581 - val_loss: 0.9738
Epoch 2/10
32000/32000 [==============================] - 204s 6ms/step - loss: 0.6277 - val_loss: 1.0010
Epoch 3/10
32000/32000 [==============================] - 205s 6ms/step - loss: 0.6126 - val_loss: 1.0112
Epoch 4/10
32000/32000 [==============================] - 205s 6ms/step - loss: 0.6039 - val_loss: 1.0122
Epoch 5/10
32000/32000 [==============================] - 205s 6ms/step - loss: 0.5960 - val_loss: 1.0288
Epoch 6/10
32000/32000 [==============================] - 205s 6ms/step - loss: 0.5888 - val_loss: 1.0235
Epoch 7/10
32000/32000 [==============================] - 204s 6ms/step - loss: 0.5830 - val_loss: 1.0361
Epoch 8/10
32000/32000 [==============================] - 204s 6ms/step - loss: 0.5773 - val_loss: 1.0425
Epoch 9/10
32000/32000 [==============================] - 204s 6ms/step - loss: 0.5724 -

In [0]:
model.reset_states()

g)

In [314]:


from keras.preprocessing import sequence


def predict_next_word(model, example, diversity=1.0):   
    preds = model.predict(np.array([example]), verbose=0)[0]
    #y_preds = [indices_vocabA[np.argmax(word)] for word in preds]
    return preds

  
  
  
n=10
for i in range(n):
    indexs = np.random.randint(0,len(Xtest_question_padded))
    example = Xtest_question_padded[indexs]
    #print(type(example))
    indexes_answer = predict_next_word(model,example,0.85)
    question = df_test["question"][indexs]
    print("Pregunta: ",question)
    answer = ""
    for index in indexes_answer:
        if indices_vocabA[np.argmax(index)]=="#end": # el final de la oracion
            continue
        else:
            answer+=indices_vocabA[np.argmax(index)]+" "
    print("Respuesta: ",answer)
print("Los ha predecido todos!")
  
  



Pregunta:  What are peptides that help microbes flourish called?
Respuesta:  the of 
Pregunta:  During what years was the Model United Nations team ranked first in South America?
Respuesta:  the and the 
Pregunta:  What did Karlen and Singer present to the US senate?
Respuesta:  the Motors 
Pregunta:  What are ctenophora commonly known as?
Respuesta:  the 11.7 
Pregunta:  How many seats must a political party have to be represented on the Parliamentary Bureau?
Respuesta:  the Domestic 
Pregunta:  Humans in tropical environments were considered what?
Respuesta:  the of 
Pregunta:  What was a long term goal of French foreign policy along the Rhine?
Respuesta:  the Cramer 
Pregunta:  Of what does the covalent double bond result from ?
Respuesta:  the the 
Pregunta:  When was the Parental Leave directive rejected?
Respuesta:  the suppression Watch 
Pregunta:  What is the Rhine called in Dutch?
Respuesta:  the Britannica 
Los ha predecido todos!


Como se dijo anteriormente es notoria la falta de entrenamiento de la red, ya que en todas las preguntas utilizadas para experimentar respondió siempre con un "the" para comenzar la frase. También se observa que las respuestas debiesen comenzar con mayúscula ya que al preprocesar no se quitó esta característica, sin embargo la red no inicia en ningun caso (en las preguntas de prueba) con palabras con primera letra mayúscula.

In [315]:

from google.colab import files
uploaded = files.upload()

Saving evaluate-v2.0.py to evaluate-v2.0.py


In [316]:
from google.colab import files
uploaded = files.upload()

Saving dev-v2.0.json to dev-v2.0.json


In [322]:
import json

dic_predictions = {}
contador = 0
for example,id_e in zip(Xtest_question_padded,df_test["id"]): #todos los ejemplos
    indexes_answer = predict_next_word(model,example) #predice palabra en cada instante
    answer = ""
    for index in indexes_answer:
        if indices_vocabA[np.argmax(index)]=="#end": # el final de la oracion
            continue
        else:
            answer+=indices_vocabA[np.argmax(index)]+" "
    dic_predictions[id_e] = answer
    contador+=1
print("Los ha predecido todos!")
print("contador: ", contador)
json_save = json.dumps(dic_predictions)
archivo = open("predictions","w")
archivo.write(json_save)
archivo.close()

Los ha predecido todos!
contador:  11873


In [323]:
#evaluar resultados
!python evaluate-v2.0.py dev-v2.0.json predictions

{
  "exact": 24.87997978606923,
  "f1": 25.334066600912525,
  "total": 11873,
  "HasAns_exact": 0.0,
  "HasAns_f1": 0.9094758354646352,
  "HasAns_total": 5928,
  "NoAns_exact": 49.68881412952061,
  "NoAns_f1": 49.68881412952061,
  "NoAns_total": 5945
}


Se observa que la red respondio un poco menos de la mitad de las preguntas de prueba, obteniendo un f1 score de 25.